# Multiprocessing in Python

Multiprocessing allows you to run multiple processes concurrently, taking advantage of multiple CPU cores for parallel processing. Here's a detailed explanation with examples:

### Basic Multiprocessing Example



In [ ]:
from multiprocessing import Process
import os
import time

def worker():
    # Get the current process ID
    process_id = os.getpid()
    # Print process information
    print(f"Worker process ID: {process_id}")
    # Simulate some work
    time.sleep(2)

def main():
    # Create a list to store processes
    processes = []
    
    # Create and start 3 worker processes
    for _ in range(3):
        # Create a new process that will run the worker function
        p = Process(target=worker)
        # Start the process
        p.start()
        # Add process to our list
        processes.append(p)
    
    # Wait for all processes to complete
    for p in processes:
        p.join()

if __name__ == '__main__':
    main()



### Process Pool Example



In [ ]:
from multiprocessing import Pool
import time

def calculate_square(number):
    # Simulate complex computation
    time.sleep(1)
    # Return square of the number
    return number * number

def main():
    # Numbers to process
    numbers = [1, 2, 3, 4, 5]
    
    # Create a pool of 3 worker processes
    with Pool(processes=3) as pool:
        # Map the calculate_square function to numbers
        # This distributes the work across the process pool
        results = pool.map(calculate_square, numbers)
        
        # Print results
        print(f"Results: {results}")

if __name__ == '__main__':
    main()



### Sharing Data Between Processes



In [ ]:
from multiprocessing import Process, Queue, Value, Array
import time

def producer(queue, counter):
    # Producer process that adds items to the queue
    for i in range(5):
        # Increment shared counter
        with counter.get_lock():
            counter.value += 1
        # Add item to queue
        queue.put(f"Item {i}")
        time.sleep(1)

def consumer(queue, counter):
    # Consumer process that reads items from the queue
    while True:
        # Try to get item from queue
        try:
            # Get item with timeout
            item = queue.get(timeout=3)
            print(f"Consumed {item}, Counter: {counter.value}")
        except:
            # Break if queue is empty for 3 seconds
            break

def main():
    # Create a shared queue for communication
    queue = Queue()
    
    # Create a shared counter
    counter = Value('i', 0)
    
    # Create producer and consumer processes
    p = Process(target=producer, args=(queue, counter))
    c = Process(target=consumer, args=(queue, counter))
    
    # Start both processes
    p.start()
    c.start()
    
    # Wait for both processes to complete
    p.join()
    c.join()

if __name__ == '__main__':
    main()



### Process Pool with Async Results



In [ ]:
from multiprocessing import Pool
import time

def long_task(name):
    # Simulate a long-running task
    time.sleep(2)
    return f"Task {name} completed"

def main():
    # Create a pool of 4 worker processes
    with Pool(processes=4) as pool:
        # Submit tasks asynchronously
        results = []
        for i in range(8):
            # Apply tasks asynchronously
            async_result = pool.apply_async(long_task, args=(i,))
            results.append(async_result)
        
        # Get results as they complete
        for result in results:
            print(result.get())  # .get() will block until result is ready

if __name__ == '__main__':
    main()



### Key Points to Remember:

1. **Process Creation**:
   - Use `Process` class for individual processes
   - Use `Pool` class for managing multiple worker processes

2. **Data Sharing**:
   - Use `Queue` for inter-process communication
   - Use `Value` and `Array` for shared memory
   - Processes don't share memory by default

3. **Best Practices**:
   - Always protect the entry point with `if __name__ == '__main__':`
   - Use `join()` to wait for processes to complete
   - Properly close and clean up resources

4. **When to Use Multiprocessing**:
   - CPU-bound tasks
   - Tasks that can run independently
   - When you need to bypass the Global Interpreter Lock (GIL)

5. **Common Issues to Watch For**:
   - Deadlocks in process communication
   - Resource consumption with too many processes
   - Proper error handling across processes

Remember that multiprocessing is best suited for CPU-intensive tasks that can run in parallel. For I/O-bound tasks, consider using asyncio or threading instead.